# Sunshine Act Cleanup
## Authors: 
    1. Lam Ho
    2. Jonah Breslow
    3. Jeffrey Kagan
## Purpose:
The purpose of this notebook is to do some preliminary cleanup/filtering of the [Center for Medicare & Medicaid Services](https://openpaymentsdata.cms.gov) data. Primarily, filter the data to just California to limit the amount of data we need to load into the Neo4J graph.

### Importing modules

In [1]:
from dask import dataframe as dd
import re

In [2]:
cols = ['Covered_Recipient_Type','Teaching_Hospital_CCN','Teaching_Hospital_ID','Teaching_Hospital_Name',
       'Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name','Physician_Last_Name','Physician_Name_Suffix',
       'Recipient_City','Recipient_State','Physician_Primary_Type','Physician_Specialty','Physician_License_State_code1','Physician_License_State_code2',
       'Physician_License_State_code3','Physician_License_State_code4','Physician_License_State_code5',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Total_Amount_of_Payment_USDollars','Date_of_Payment','Number_of_Payments_Included_in_Total_Amount',
       'Form_of_Payment_or_Transfer_of_Value','Nature_of_Payment_or_Transfer_of_Value','City_of_Travel',
       'State_of_Travel', 'Country_of_Travel','Physician_Ownership_Indicator','Third_Party_Payment_Recipient_Indicator',
       'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value','Charity_Indicator','Third_Party_Equals_Covered_Recipient_Indicator',
       'Record_ID','Related_Product_Indicator','Covered_or_Noncovered_Indicator_1',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1',
       'Product_Category_or_Therapeutic_Area_1','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
       'Associated_Drug_or_Biological_NDC_1','Covered_or_Noncovered_Indicator_2',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Product_Category_or_Therapeutic_Area_2',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Associated_Drug_or_Biological_NDC_2',
       'Covered_or_Noncovered_Indicator_3',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Product_Category_or_Therapeutic_Area_3',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Associated_Drug_or_Biological_NDC_3',
       'Covered_or_Noncovered_Indicator_4',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Product_Category_or_Therapeutic_Area_4',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Associated_Drug_or_Biological_NDC_4',
       'Covered_or_Noncovered_Indicator_5',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5',
       'Product_Category_or_Therapeutic_Area_5',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
       'Associated_Drug_or_Biological_NDC_5','Recipient_Zip_Code']

In [3]:
#Load the data
sunshine = dd.read_csv('Data/OP_DTL_GNRL_PGYR2019_P06302021.csv',dtype=str,usecols=cols)

## Pre-Processing Text Columns

In [4]:
def remove_punc(string):
    '''
    takes in a string and removed all punctuation.
    '''
    return re.sub(r'[^\w\s]','',string)

def preProcess(df):
    '''
    takes a dataframe, applies the remove_punc function
    and then upper cases all text columns
    '''
    df = df.applymap(lambda s:remove_punc(s).upper() if isinstance(s, str) else s)
    return df

### CA Filter
1. Filter only for California 
2. Persisting `cadf` in memory so we don't need to repeat this filtering step every time we want to create a new output file

In [5]:
%%time
cadf = sunshine[sunshine['Recipient_State']=='CA']
cadf = cadf.map_partitions(preProcess)
cadf = cadf.persist()
del sunshine

Wall time: 3min 39s


### Creating Hospital-Payment edge

In [6]:
%%time
# Hospital ID and Payment ID
hosp = cadf[cadf['Covered_Recipient_Type']=='COVERED RECIPIENT TEACHING HOSPITAL'][['Record_ID','Teaching_Hospital_ID']]
hosp.to_csv('Data/Outputs_Cleanup/Sunshine/hospitals_payment.csv',single_file=True,index=False)

Wall time: 4.21 s


['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/hospitals_payment.csv']

### Creating Hospital-State edge

In [7]:
%%time
# Hospitals and states
hosp = cadf[cadf['Covered_Recipient_Type']=='COVERED RECIPIENT TEACHING HOSPITAL'][['Recipient_State','Teaching_Hospital_ID']]
hosp = hosp.drop_duplicates(subset=['Teaching_Hospital_ID'])

Wall time: 34 ms


In [8]:
hospital_state_dupes = hosp.groupby('Teaching_Hospital_ID').count().compute()
hosp_state_dupes = hospital_state_dupes[hospital_state_dupes.Recipient_State>1]
print("There are "+str(hosp_state_dupes.shape[0])+ " hospitals with more than 1 associated state")

There are 0 hospitals with more than 1 associated state


In [9]:
hosp.to_csv('Data/Outputs_Cleanup/Sunshine/hosp_states.csv',single_file=True,index=False)

['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/hosp_states.csv']

### Crating Hospital Node

In [10]:
%%time
# Hospitals' other information
hosp = cadf[cadf['Covered_Recipient_Type']=='COVERED RECIPIENT TEACHING HOSPITAL'][['Teaching_Hospital_CCN',
                                                                                    'Teaching_Hospital_ID',
                                                                                    'Teaching_Hospital_Name']]
hosp = hosp.drop_duplicates(subset=['Teaching_Hospital_ID'])

Wall time: 5.03 ms


In [11]:
hospital_node_dupes = hosp.groupby('Teaching_Hospital_ID').count().compute()
hosp_node_dupes = hospital_node_dupes[hospital_node_dupes.Teaching_Hospital_CCN>1]
print("There are "+str(hosp_node_dupes.shape[0])+ " hospitals with more than 1 associated state")

There are 0 hospitals with more than 1 associated state


In [12]:
hosp.to_csv('Data/Outputs_Cleanup/Sunshine/hosp_info.csv',single_file=True,index=False)
del hosp

### Creating Physician-Payment edge

In [13]:
%%time
# Physician ID and Payment ID
physicians = cadf[cadf['Covered_Recipient_Type']=='COVERED RECIPIENT PHYSICIAN'][['Record_ID','Physician_Profile_ID']]
physicians.to_csv('Data/Outputs_Cleanup/Sunshine/physicians_payments.csv',single_file=True,index=False)

Wall time: 20.3 s


['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/physicians_payments.csv']

### Creating Physician-State edge

In [14]:
%%time
# Physicians and states
physicians = cadf[cadf['Covered_Recipient_Type']=='COVERED RECIPIENT PHYSICIAN'][['Recipient_State','Physician_Profile_ID']]
physicians = physicians.drop_duplicates(subset=['Physician_Profile_ID'])

Wall time: 54.8 ms


In [15]:
physician_state_dupes = physicians.groupby('Physician_Profile_ID').count()
physician_state_dupes = physician_state_dupes[physician_state_dupes.Recipient_State>1].compute()
print("There are "+str(physician_state_dupes.shape[0])+ " physcians with more than 1 associated state")

There are 0 physcians with more than 1 associated state


In [16]:
%%time
# Physicians and states
physicians = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Physician'][['Recipient_State','Physician_Profile_ID']]
physicians = physicians.drop_duplicates(subset=['Physician_Profile_ID'])

Wall time: 26 ms


### Physician Node

In [17]:
physicians.to_csv('Data/Outputs_Cleanup/Sunshine/physicians_state.csv',single_file=True,index=False)

['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/physicians_state.csv']

### Creating Physician Node

In [18]:
%%time
# Physicians other information
physicians = cadf[cadf['Covered_Recipient_Type']=='COVERED RECIPIENT PHYSICIAN'][['Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name','Physician_Last_Name','Physician_Name_Suffix',
       'Recipient_City','Recipient_State','Physician_Primary_Type','Physician_Specialty','Physician_License_State_code1','Physician_License_State_code2',
       'Physician_License_State_code3','Physician_License_State_code4','Physician_License_State_code5','Recipient_Zip_Code']]
physicians = physicians.drop_duplicates(subset=['Physician_Profile_ID'])

Wall time: 12 ms


In [19]:
physician_node_dupes = physicians.groupby('Physician_Profile_ID').count()
physician_node_dupes =physician_node_dupes[physician_node_dupes.Physician_First_Name>1].compute()
print("There are "+str(physician_node_dupes.shape[0])+ " physcians with duplicated node data")

There are 0 physcians with duplicated node data


In [20]:
physicians.to_csv('Data/Outputs_Cleanup/Sunshine/physicians_info.csv',single_file=True,index=False)
del physicians

### Creating Payment Node

In [21]:
%%time
# Payment information
payments = cadf[['Record_ID','Total_Amount_of_Payment_USDollars','Date_of_Payment','Number_of_Payments_Included_in_Total_Amount',
                 'Form_of_Payment_or_Transfer_of_Value','Nature_of_Payment_or_Transfer_of_Value']]
payments.to_csv('Data/Outputs_Cleanup/Sunshine/payments.csv',single_file=True,index=False)
del payments

Wall time: 6.3 s


### Creating PharmCo-Payment Edge

In [22]:
%%time
# Pharmco and Record ID
pharmCo = cadf[['Record_ID','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID']]
pharmCo.to_csv('Data/Outputs_Cleanup/Sunshine/pharmCo_payments.csv',single_file=True,index=False)

Wall time: 3.43 s


['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/pharmCo_payments.csv']

### Creating PharmCo-State Edge

In [23]:
%%time
# Pharmco and state
pharmCo = cadf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State']]
pharmCo = pharmCo.drop_duplicates()

Wall time: 24.6 ms

In [24]:
pharmco_state_dupes = pharmCo.groupby('Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID').count()
pharmco_state_dupes = pharmco_state_dupes[pharmco_state_dupes.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State>1].compute()
print("There are "+str(pharmco_state_dupes.shape[0])+ " PharmCo's with multiple states")

There are 0 PharmCo's with multiple states


In [25]:
pharmCo.to_csv('Data/Outputs_Cleanup/Sunshine/pharmCo_states.csv',single_file=True,index=False)

['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/pharmCo_states.csv']

### Creating PharmCo Node
Interesting note: `Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name` is not always the same for the same `Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID`. To handle this, we are just selecting the first one. There are only a couple dupes and they are clearly the same PharmaCo

In [26]:
%%time
# Pharmco other information
pharmCo = cadf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
#                 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
                'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name']].drop_duplicates('Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID')

Wall time: 3.04 ms


In [27]:
pharmco_node_dupes = pharmCo.groupby('Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID').count()
pharmco_node_dupes = pharmco_node_dupes[pharmco_node_dupes.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name>1].compute()
print("There are "+str(pharmco_node_dupes.shape[0])+ " PharmCo's with duplicate node data")

There are 0 PharmCo's with duplicate node data


In [28]:
pharmCo.to_csv('Data/Outputs_Cleanup/Sunshine/pharmCo_info.csv',single_file=True,index=False)

['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/pharmCo_info.csv']

In [29]:
%%time
# Pharmco other information
pharmCo = cadf[['Record_ID','Covered_or_Noncovered_Indicator_1','Related_Product_Indicator',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1','Product_Category_or_Therapeutic_Area_1',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1','Associated_Drug_or_Biological_NDC_1','Covered_or_Noncovered_Indicator_2',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2','Product_Category_or_Therapeutic_Area_2',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2','Associated_Drug_or_Biological_NDC_2',
                'Covered_or_Noncovered_Indicator_3','Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3',
                'Product_Category_or_Therapeutic_Area_3','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
                'Associated_Drug_or_Biological_NDC_3','Covered_or_Noncovered_Indicator_4',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4','Product_Category_or_Therapeutic_Area_4',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4','Associated_Drug_or_Biological_NDC_4',
                'Covered_or_Noncovered_Indicator_5','Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5',
                'Product_Category_or_Therapeutic_Area_5','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
                'Associated_Drug_or_Biological_NDC_5']]
pharmCo.to_csv('Data/Outputs_Cleanup/Sunshine/record_drugs.csv',single_file=True,index=False)

Wall time: 14 s


['C:/Users/10294029/Documents/GitHub/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/Sunshine/record_drugs.csv']